In [1]:
# external imports
import music21 as m21
import ast
import os

In [14]:
# reload cell
from src.midi import *
from src.train import *
import sys, importlib
importlib.reload(sys.modules['src.midi'])
importlib.reload(sys.modules['src.train'])

<module 'src.train' from 'D:\\Documents\\GitHub\\Pyotr\\src\\train.py'>

# Encoding and Read-in

In [10]:
# Read in all of the midi files
path = './data/sample'
mdl = gen_md_from_path(path, by_measure=False, verbose=False)
mdm = gen_md_from_path(path, by_measure=True, verbose=False)

In [11]:
# Encode them (as entire piece)
me = MidiEncoder()
mdl_enc = {}
for piece in mdl:
    mdl_enc[piece] = me.Encode(mdl[piece].flat, 'pitch_position_duration_strings')

In [12]:
# Encode them (by measure)
me = MidiEncoder()
mdm_enc = {}
for piece in mdm:
    mdm_enc[piece] = {}
    for i, m in enumerate(mdm[piece]):
        mdm_enc[piece][i] = me.Encode(m, 'pitch_position_duration_strings')

# Pre-processing

In [15]:
data = TrainingSet(mdm_enc, by_measure=True, num_notes=32, build_type='next_note')

In [18]:
data.get_vocab_size()

589

# Modeling

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# Params

In [ ]:
# Sample network
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores